In [1]:
import os
from langchain_community.document_loaders import UnstructuredFileLoader
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_openai import ChatOpenAI
from langchain_community.graphs import Neo4jGraph
from langchain.docstore.document import Document
from langchain.chains import GraphQAChain
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from pydantic import Field
from langchain.chains.base import Chain
from typing import Dict, Any, List

In [3]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["NEO4J_URI"] = os.getenv("NEO4J_URI")
os.environ["NEO4J_USERNAME"] = os.getenv("NEO4J_USERNAME")
os.environ["NEO4J_PASSWORD"] = os.getenv("NEO4J_PASSWORD")

In [4]:
loader = UnstructuredFileLoader("/Users/siddharthdileep/extracter/01471587/shortened-filings/AA_MzE1ODIxMjk1NmFkaXF6a2N4.pdf")
docs = loader.load()

/var/folders/qw/2ff82h_n6076ds23_xr8_9q40000gn/T/ipykernel_5063/1621351887.py:1: LangChainDeprecationWarning: The class `UnstructuredFileLoader` was deprecated in LangChain 0.2.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-unstructured package and should be used instead. To use it run `pip install -U :class:`~langchain-unstructured` and import as `from :class:`~langchain_unstructured import UnstructuredLoader``.
  loader = UnstructuredFileLoader("/Users/siddharthdileep/extracter/01471587/shortened-filings/AA_MzE1ODIxMjk1NmFkaXF6a2N4.pdf")
/Users/siddharthdileep/extracter/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
type(docs)
docs[0].page_content

'{mem\n\nCOMPANIES HOUSE\n\nVODAFONE LIMITED Company No: 01471587\n\nSTRATEGIC REPORT (continued)\n\nStrategic report for the year ended 31 March 2016 The directors present their strategic report for the year ended 31 March 2016. Review of the business and future developments\n\nThe overall strategy of Vodafone Limited ("the Company”) is aligned to that of Vodafone Group Plc (“the Group”), which is outlined in the Group Strategy Review in the Group\'s Annual Report and Financial Statements for the year ended 31 March 2016, which does not form part of this report.\n\nThe Company has transitioned to FRS 101 — Reduced Disclosure Framework for the preparation of these financial statements. Comparative information has been restated as appropriate; see note 1 to the financial statements for the impact of the adoption and the quantification of the historical adjustments.\n\nKey financial performance indicators for the Company include revenue, operating loss and average revenue per user (“ARPU

In [11]:
llm = ChatOpenAI(temperature=0, model_name="gpt-4o-mini")
llm_transformer = LLMGraphTransformer(llm=llm, node_properties=True)


In [ ]:
graph_documents = llm_transformer.convert_to_graph_documents(docs)

In [15]:
NEO4J_URI = os.getenv("NEO4J_URI")
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")

In [13]:
graph = Neo4jGraph(
    url=os.getenv("NEO4J_URI"),
    username=os.getenv("NEO4J_USERNAME"),
    password=os.getenv("NEO4J_PASSWORD")
)

/var/folders/qw/2ff82h_n6076ds23_xr8_9q40000gn/T/ipykernel_5063/2731935992.py:1: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph = Neo4jGraph(


In [ ]:
graph.add_graph_documents(graph_documents=graph_documents, baseEntityLabel=True, include_source=True)

In [14]:
graph.refresh_schema()


In [16]:
from langchain_neo4j import Neo4jVector
from langchain.embeddings import OpenAIEmbeddings

vector_index = Neo4jVector.from_existing_graph(
    OpenAIEmbeddings(),
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding",
)

# Now embed and store vectors
vector_index.add_documents(docs)

/var/folders/qw/2ff82h_n6076ds23_xr8_9q40000gn/T/ipykernel_5063/2097507220.py:5: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  OpenAIEmbeddings(),


['52b2760d775255f59e0b062a33b32e9c']

In [17]:
results = vector_index.similarity_search(
    "What is the name of the company", 
    k=3
)

for i, doc in enumerate(results, 1):
    print(f"\n--- Result {i} ---")
    print(doc.page_content)
    print(doc.metadata)


--- Result 1 ---

text: COMPANIES FORM No. 123
G
Notice of increase
123
in nominal capital
CHFP055
Please do not
Pursuant to section 123 of the Companies Act 1985
write in this
margin
To the Registrar of Companies
Please complete
legibly, preferably (Address overleaf)
in black type, or
For official use
Company Number
bold block
lettering
1471587
Name of Company
insert full name
* VODAFONE LIMITED
of company
gives notice in accordance with section 123 of the above Act that by resolution of the company
§ the copy must
be printed or in
dated 31 August 2000 the nominal capital of the company has been increased by
some other form
£9,999,999,000.000 beyond the registered capital of £1,000.000.
approved by the
A copy of the resolution authorising the increase is attached. §
registrar
The conditions (e.g. voting rights, dividend rights, winding-up rights etc.) subject to which the
new shares have been or are to be issued are as follows:
Out of the profits available for distribution and resolv

In [18]:
results

[Document(metadata={}, page_content='\ntext: COMPANIES FORM No. 123\nG\nNotice of increase\n123\nin nominal capital\nCHFP055\nPlease do not\nPursuant to section 123 of the Companies Act 1985\nwrite in this\nmargin\nTo the Registrar of Companies\nPlease complete\nlegibly, preferably (Address overleaf)\nin black type, or\nFor official use\nCompany Number\nbold block\nlettering\n1471587\nName of Company\ninsert full name\n* VODAFONE LIMITED\nof company\ngives notice in accordance with section 123 of the above Act that by resolution of the company\n§ the copy must\nbe printed or in\ndated 31 August 2000 the nominal capital of the company has been increased by\nsome other form\n£9,999,999,000.000 beyond the registered capital of £1,000.000.\napproved by the\nA copy of the resolution authorising the increase is attached. §\nregistrar\nThe conditions (e.g. voting rights, dividend rights, winding-up rights etc.) subject to which the\nnew shares have been or are to be issued are as follows:\nOu

In [ ]:
from pyvis.network import Network
import networkx as nx

def showGraph(cypher: str = default_cypher):
    driver = GraphDatabase.driver(
        uri=os.environ["NEO4J_URI"],
        auth=(os.environ["NEO4J_USERNAME"], os.environ["NEO4J_PASSWORD"])
    )
    with driver.session() as session:
        results = session.run(cypher)
        G = nx.Graph()
        for record in results:
            s = record["s"]
            t = record["t"]
            G.add_node(s.id)
            G.add_node(t.id)
            G.add_edge(s.id, t.id)
        net = Network(notebook=True)
        net.from_nx(G)
        net.show("graph.html")
    driver.close()

In [19]:
graph.refresh_schema()
from langchain.prompts import PromptTemplate
from langchain.chains import GraphCypherQAChain
CYPHER_GENERATION_TEMPLATE = """
You are an expert in Cypher query language for Neo4j.
Given the following schema:

{schema}

Please write a valid Cypher query to answer this question:
{question}

Return ONLY the Cypher query without any explanations or text before or after.
"""
CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question"], template=CYPHER_GENERATION_TEMPLATE
)

CYPHER_QA_TEMPLATE = """You are an assistant that helps to form nice and human understandable answers.
...
"""
CYPHER_QA_PROMPT = PromptTemplate(
    input_variables=["context", "question"], template=CYPHER_QA_TEMPLATE
)

graph_chain = GraphCypherQAChain.from_llm(
    ChatOpenAI(temperature=0, model_name="gpt-4o-mini"), 
    graph=graph, 
    cypher_prompt=CYPHER_GENERATION_PROMPT, 
    qa_prompt=CYPHER_QA_PROMPT,
    verbose=True,
    allow_dangerous_requests=True
)

In [27]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate
from pydantic import Field
from langchain.chains.base import Chain
from typing import Dict, Any, List

# This custom chain class combines the results from both retrievers
class HybridRetrieval(Chain):
    """Chain that combines graph and vector search results."""
    
    graph_chain: Chain = Field(...)
    vector_index: Any = Field(...)
    
    @property
    def input_keys(self) -> List[str]:
        return ["query"]
    
    @property
    def output_keys(self) -> List[str]:
        return ["result"]
    
    def _call(self, inputs: Dict[str, Any]) -> Dict[str, Any]:
        question = inputs["query"]
        
        try:
            # Get results from the Cypher QA chain
            graph_result = self.graph_chain.invoke(question)
            graph_text = graph_result.get('result', "No graph results found")
        except Exception as e:
            graph_text = f"Graph query error: {str(e)[:100]}"
        
        # Get results from the vector retriever
        vector_docs = self.vector_index.similarity_search(question, k=3)
        vector_result = "\n\n".join([doc.page_content for doc in vector_docs])
        
        # Combine the results into a single string
        combined_result = f"Graph Result:\n{graph_text}\n\nVector Results:\n{vector_result}"
        print(f"Combined Result:\n{combined_result}")
        return {"result": combined_result}

# Initialize the hybrid retrieval chain
hybrid_retriever = HybridRetrieval(
    graph_chain=graph_chain,  # Your existing GraphCypherQAChain
    vector_index=vector_index  # Your existing Neo4jVector
)

# Create the final question-answering chain using the hybrid retriever
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
final_rag_chain = {
    "context": lambda x: hybrid_retriever.run(x),
    "question": lambda x: x
}
final_rag_chain = {
    "context": lambda x: hybrid_retriever.invoke(x)["result"],
    "question": lambda x: x
} | prompt | ChatOpenAI(temperature=0, model="gpt-4o-mini") | StrOutputParser()

# Example usage
query = "can you give me brief summary while highlighting financial terms and values talked in this document"
result = final_rag_chain.invoke(query)





> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (d:Document)-[:MENTIONS]->(c:Capital) 
RETURN d.text AS documentText, collect(c.id) AS mentionedCapital
UNION
MATCH (d:Document)-[:MENTIONS]->(dr:Dividend rate) 
RETURN d.text AS documentText, collect(dr.id) AS mentionedDividendRates
UNION
MATCH (d:Document)-[:MENTIONS]->(l:Legislation) 
RETURN d.text AS documentText, collect(l.id) AS mentionedLegislation
UNION
MATCH (d:Document)-[:MENTIONS]->(a:Address) 
RETURN d.text AS documentText, collect(a.id) AS mentionedAddresses
UNION
MATCH (d:Document)-[:MENTIONS]->(s:Share class) 
RETURN d.text AS documentText, collect(s.id) AS mentionedShareClasses

Combined Result:
Graph Result:
Graph query error: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input 'rate': expected a parameter, 

Vector Results:

text: {mem

COMPANIES HOUSE

VODAFONE LIMITED Company No: 01471587

STRATEGIC REPORT (continued)

Strategic report for the year ended 31 March 2016 The d

In [28]:
result

"The document is a strategic report for Vodafone Limited for the year ended 31 March 2016, detailing the company's financial performance, position, and future outlook. Here are the key financial terms and values highlighted:\n\n1. **Revenue**: Total revenue for the year was £6,183.5 million, a decline of 1.8% from £6,294.2 million in the previous year.\n\n2. **Operating Loss**: The company incurred an operating loss of £354.8 million, compared to a loss of £308.9 million in 2015.\n\n3. **Loss for the Financial Year**: The total loss for the year was £658.2 million, up from a loss of £536.9 million in 2015.\n\n4. **Average Revenue Per User (ARPU)**: Contract ARPU decreased to £26.50 from £27.20 in the previous year.\n\n5. **Cash Position**: The company closed the year with cash of £19.0 million, down from €63.3 million in 2015.\n\n6. **Debt Management**: In March 2016, Vodafone repaid a loan note through the issuance of new share capital, reducing annual interest expenses by over €300 m